# Combine bcgov stats
This notebook grabs bcgov stats from different csvs and combines them into one table.

In [1]:
import pandas as pd
import numpy as np
from functions import parse_file, left_merge, count_repositories, format_repolint_results, print_to_csv
pd.set_option('display.max_columns', None)

In [2]:
# Import csvs to be combined
df = parse_file('repository-basics.csv', 'bcgov')
issues_df = parse_file('issues.csv', 'bcgov', df)
topics_df = parse_file('repo-topics.csv', 'bcgov', df)
df_pr = parse_file('repository-pr.csv', 'bcgov', df)
df_repo_1 = parse_file("repository-details-1.csv", 'bcgov', df)
df_repo_2 = parse_file("repository-details-2.csv", 'bcgov', df)
repolint_df = parse_file('repolint-results.csv', 'bcgov', df)

## Issues
The issues table contains information about all issues in the organization, and is one to many. For information about repomountie issues that aren't closed, we can grab all issues by the author with relevant titles that are open, and aggregate them to get a total count.

In [3]:
def add_repomountie_issue_count(titles, issues_df, master):
    merged_df = master
    for title in titles:
        open_issues = issues_df.loc[
            (issues_df['Author'] == 'repo-mountie') &
            (issues_df["Title"] == title) &
            (issues_df["State"] == "OPEN")
        ]
        open_issues_count = count_repositories(open_issues, 'Open {title} issues'.format(title=title))
        merged_df = left_merge(merged_df, open_issues_count)
    return merged_df


In [4]:
# Run function for each rm title
merged_df = add_repomountie_issue_count([
 "Add project lifecycle badge",
 "Lets use common phrasing",
 "Add missing topics",
], issues_df, df)

# Add total RM issues open
all_open_rm_issues = issues_df.loc[
    (issues_df['Author'] == 'repo-mountie') &
    (issues_df["State"] == "OPEN")
]
total_open_rm_issues = count_repositories(all_open_rm_issues, "Open Repomountie Issues")
merged_df = left_merge(merged_df, total_open_rm_issues)

# Since counts are numbers, fill NaNs with 0 for these columns
merged_df.fillna({'Open Add missing topics issues':0, 'Open Lets use common phrasing issues':0, "Open Add project lifecycle badge issues": 0, "Open Repomountie Issues": 0}, inplace=True)

## Topics

The topics csv contains all ministry topic codes for a repository, and also needs to be aggregated to get the total count of ministry topics.

In [5]:
topics_count = topics_df.groupby('Repository').agg("count").reset_index()
merged_df = left_merge(merged_df, topics_count[["Repository", "Ministry Code"]])
merged_df.rename(columns = {
    "Ministry Code": "Ministry Codes Count"
}, inplace=True)

# Repolint Results
The results from the repolint need some light formatting to convert PASSED and NOT_PASSED strings into booleans, and remove some columns that aren't necessary in the combined sheet.

In [6]:
formatted_repolint_results = format_repolint_results(repolint_df)
merged_df = left_merge(merged_df, formatted_repolint_results)

The remaining csvs can be merged directly

In [7]:
merged_df = left_merge(merged_df, df_repo_1)
merged_df = left_merge(merged_df, df_repo_2)
merged_df = left_merge(merged_df, df_pr)

In [8]:
merged_df.info()
merged_df.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 62 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Repository                               975 non-null    object 
 1   owner                                    975 non-null    object 
 2   name_with_owner                          975 non-null    object 
 3   url                                      975 non-null    object 
 4   homepage_url                             130 non-null    object 
 5   description                              825 non-null    object 
 6   forks                                    975 non-null    int64  
 7   is_fork                                  975 non-null    bool   
 8   is_mirror                                975 non-null    bool   
 9   is_private                               975 non-null    bool   
 10  is_archived                              975 non-n

,Repository,owner,name_with_owner,url,homepage_url,description,forks,is_fork,is_mirror,is_private,is_archived,is_template,stars,disk_usage,has_issues_enabled,has_projects_enabled,has_wiki_enabled,merge_commit_allowed,rebase_merge_allowed,squash_merge_allowed,created_at,updated_at,pushed_at,Open Add project lifecycle badge issues,Open Lets use common phrasing issues,Open Add missing topics issues,Open Repomountie Issues,Ministry Codes Count,license-file-exists,readme-file-exists,contributing-file-exists,code-of-conduct-file-exists,changelog-file-exists,security-file-exists,support-file-exists,readme-references-license,binaries-not-present,test-directory-exists,integrates-with-ci,code-of-conduct-file-contains-email,source-license-headers-exist,github-issue-template-exists,github-pull-request-template-exists,PackageCount,ProjectCount,ReleaseCount,SubmoduleCount,DeployKeyCount,TopicCount,License,CodeOfConduct,DaysOpen,IssueCount,PrCount,CommitCount,AverageIssueCountPerDay,AveragePrCountPerDay,AverageCommitCountPerDay,DefaultBranchName,Languages,Fork PullRequest Count,Review Count
0,wqbc,bcgov,bcgov/wqbc,https://github.com/bcgov/wqbc,http://bcgov.github.io/wqbc/,An R package for water quality thresholds and ...,9,False,False,False,False,False,14,26448,True,True,True,True,True,True,2014-12-15 19:26:57,2021-01-27 13:45:38,2020-12-16 15:14:54,0.0,0.0,0.0,0.0,1.0,True,True,True,True,False,False,False,True,True,True,True,True,False,False,False,0.0,0.0,9.0,0.0,0.0,5.0,Apache License 2.0,Contributor Covenant,2319.0,104.0,56.0,778.0,0.04,0.02,0.34,master,R_TeX_Rebol,0.0,16.0
1,BC-Policy-Framework-For-GitHub,bcgov,bcgov/BC-Policy-Framework-For-GitHub,https://github.com/bcgov/BC-Policy-Framework-F...,NaN,Policy information for BC Government employees...,43,False,False,False,False,False,101,1096,True,True,True,True,True,True,2015-02-26 15:52:57,2021-04-13 00:19:10,2021-02-19 09:56:56,1.0,0.0,0.0,1.0,1.0,True,True,True,True,False,False,False,True,True,False,False,True,False,False,False,0.0,0.0,1.0,0.0,0.0,8.0,Other,Contributor Covenant,2246.0,20.0,43.0,186.0,0.01,0.02,0.08,master,NaN,0.0,11.0


In [10]:
print_to_csv(merged_df, 'bcgov', 'master.csv')